In [11]:
import os
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
from dotenv import load_dotenv
from pathlib import Path 
import matplotlib.pyplot as plt
import psycopg2
from dash import Dash, html, dash_table, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [12]:
check_query = """
OPTIONAL MATCH (n) WITH n LIMIT 1 
RETURN n IS NOT NULL AS IsNodesPresent
"""
country_query = """
MATCH (n:country) RETURN n.code as Code, n.zone as Zone, n.name as Name LIMIT 25
"""

emission_by_country_query = """
MATCH (c:country)-[r1:UNDER_IPCC_CODE]->(i:ipcc)-[r2:HAS_PRODUCED]->(e:emission)
WITH c,i,e,r1,r2
WHERE c.code = '{key}'
RETURN
    c.code as Code,
    c.zone as Zone,
    c.name as Name,
    i.ipcc_code as IPCC_Code,
    i.ipcc_name as IPCC_Code_Name,
    e.total_emission as Total_Emission
    ORDER BY Code
"""

emission_by_country_year_query = """
MATCH (c:country)-[r1:UNDER_IPCC_CODE]->(i:ipcc)-[r2:HAS_PRODUCED]->(e:emission)
WITH c,i,e,r1,r2
WHERE i.ipcc_code = '{key1}'
RETURN
    c.code as Code,
    c.zone as Zone,
    c.name as Name,
    i.ipcc_code as IPCC_Code,
    i.ipcc_name as IPCC_Code_Name,
    e.`{key2}` as Y_{key2},
    e.total_emission as Total_Emission
    ORDER BY Code
"""

emission_by_secotrs_query = """
MATCH (c:country)-[r1:UNDER_IPCC_CODE]->(i:ipcc)-[r2:HAS_PRODUCED]->(e:emission)
WITH c,i,e,r1,r2
WHERE i.ipcc_code = '{key}'
RETURN
    c.name as Country_Name,
    i.ipcc_code as Code,
    i.ipcc_name as Name, 
    e.total_emission as Total_Emission
"""
complete_emission_query="""
MATCH (c:country)-[r1:UNDER_IPCC_CODE]->(i:ipcc)-[r2:HAS_PRODUCED]->(e:emission)
WITH c,i,e,r1,r2
RETURN
    c.code as Code,
    c.zone as Zone,
    c.name as Name,
    i.ipcc_code as IPCC_Code,
    i.ipcc_name as IPCC_Code_Name,
    properties(e) as emissions
ORDER BY Code
"""

In [ ]:
def connectToDataBases():
    env_path = Path('..') / '.env'
    if(load_dotenv(dotenv_path=env_path)):
        host = os.environ.get("neo_host")
        username = os.environ.get("neo_username")
        password = os.environ.get("neo_password")
    else:
        print(".env file does not have required values!")
    driver = GraphDatabase.driver(host,auth=(username, password))
    return driver

In [14]:
def getCountry(query,name=""):
    with driver.session() as session: 
        result = session.run(query)
        data = [dict(record) for record in result]
        return pd.DataFrame(data)
    
def getRecords(query, values=[]):
    with driver.session() as session:
        f_query = ""
        if(len(values) > 0):
            if(len(values) == 1):
                f_query = query.format(key=values[0])
            elif(len(values) == 2):
                f_query = query.format(key1=values[0],key2=values[1])
            else:
                f_query = query
        else:
            f_query = query
        result = session.run(f_query)
        data = [dict(record) for record in result]
        return pd.DataFrame(data)

In [15]:
driver = connectToDataBases()

In [16]:
emission_by_country = getRecords(emission_by_country_query,["ABW"])
emission_by_secotrs= getRecords(emission_by_secotrs_query,["1.A.3.d"])
emission_by_country_year = getRecords(emission_by_country_year_query,["1.A.3.d","2020"])
init_complete_emission = getRecords(complete_emission_query)

In [17]:
def createCompleteData(df):
    records=[]
    for i, row in df.iterrows():
        emission_body={
            'Code':row["Code"],
            'Zone':row["Zone"],
            'Name':row["Name"],
            'IPCC_Code':row["IPCC_Code"],
            'IPCC_Code_Name':row["IPCC_Code_Name"],
        }
        for year_col in sorted(col for col in row['emissions'].keys()):
            emission_body[f"{year_col}"] = row['emissions'][year_col]
        records.append(emission_body)
    return pd.DataFrame(records)

In [18]:
complete_emission = createCompleteData(init_complete_emission)

In [19]:
year_cols = [col for col in complete_emission.columns if col.isdigit()]

## PostgreSQL connection

In [ ]:
def connectToDB(db = "postgres"):
    try:
        env_path = Path('..') / '.env'
        if(load_dotenv(dotenv_path=env_path)):
            _host = os.environ.get("pg_host")
            _username = os.environ.get("pg_username")
            _password = os.environ.get("pg_password")
            _port = int(os.environ.get("pg_port"))
            conn = psycopg2.connect(database=db,
                    host=_host,
                    user=_username,
                    password=_password,
                    port=_port)
            return conn
        else:
            return None
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

In [21]:
conn = connectToDB("co2_emission")
cursor = conn.cursor()

In [22]:
all_county_query = """SELECT code FROM country"""

emission_by_year = """SELECT c.name,sum(e.Y_{year}) AS emission_rate FROM emission e 
INNER JOIN country c ON e.country_id = c.country_id GROUP BY c.name;"""

In [23]:
cursor.execute(all_county_query)
country_list = [str(country[0]) for country in cursor.fetchall()]

In [24]:
def getEmissionByYear(query,value="2000"):
    cursor.execute(query.format(year=value))
    return cursor.fetchall()

In [25]:
emissionByYear = getEmissionByYear(emission_by_year)
map_plot={
        "country":[str(x[0]) for x in emissionByYear],
        "emission":[str(x[1]) for x in emissionByYear]
}

In [31]:
plot_data = {
    "Year":year_cols,
    'Emissions':complete_emission[year_cols].sum().values
}
plot_df=pd.DataFrame(plot_data)

In [26]:
def layout_01():
    return html.Div([
    html.Div(children='My First App with Data and a Graph'),
    dcc.Graph(figure=px.histogram(plot_df, x='Year', y='Emissions', histfunc='avg')),
    dcc.Graph(figure=px.line(plot_df,x='Year',y='Emissions')),
    dcc.Dropdown(country_list, "BRA", id="country-dropdown"),
    html.Div(id="dd-output-container"),
    dcc.Graph(id="country-graph"),
    dcc.Graph(id="country-ipcc-pie-graph"),
    dcc.Dropdown([str(i) for i in range(2000,2024)], "2000", id="year-dropdown"),
    dcc.Graph(id="continent-map")],className="layout-1")

In [27]:
def callback_bar(app):
    @app.callback(
        Output('country-graph', 'figure'),
        Input('country-dropdown', 'value')
    )

    def update_bar(value):
        emission_by_country = getRecords(emission_by_country_query,[value])
        graph_df = emission_by_country[emission_by_country["Code"] == value]

        fig = px.bar(graph_df,x="IPCC_Code",y="Total_Emission")
        return fig

In [28]:
def callback_pie(app):
    @app.callback(
        Output('country-ipcc-pie-graph', 'figure'),
        Input('country-dropdown', 'value')
    )
    def update_pie(value):
        emission_by_country = getRecords(emission_by_country_query,[value])
        graph_df = emission_by_country[emission_by_country["Code"] == value]

        fig = px.pie(graph_df, values='Total_Emission', names='IPCC_Code', title=f'Emission by IPCC code for {graph_df["Name"][0]}').update_traces(textposition='inside', textinfo='percent+label')
        return fig

In [29]:
red_scale = [
    [0.0, 'rgb(103, 0, 0)'], 
    [0.2, 'rgb(178, 24, 43)'],
    [0.4, 'rgb(214, 96, 77)'],
    [0.6, 'rgb(244, 165, 130)'],
    [0.8, 'rgb(253, 219, 199)'],
    [1.0, 'rgb(255, 245, 240)']
]

In [30]:
def callback_country_emission(app):
    @app.callback(
        Output('continent-map', 'figure'),
        Input('year-dropdown', 'value')
    )
    def update_country_emission(value):
        emissionByYear = getEmissionByYear(emission_by_year,value)
        map_plot={
            "country":[str(x[0]) for x in emissionByYear],
            "emission":[int(x[1]) for x in emissionByYear]
        }
        graph_df = pd.DataFrame(map_plot)
        fig = px.choropleth(graph_df,
                        locations="country",
                        locationmode="country names",
                        color="emission",
                        scope="europe",
                        title="Emission by Country",
                        hover_name = "country",
                        hover_data = {"emission":True,"country":False},
                        labels={"emission":"Emission Rate"},
                        )
        fig.update_geos(showcountries=True, showcoastlines=True)
        fig.update_layout(height=600,coloraxis_colorbar={"title":"emission","tickformat":",.0f"})
        return fig